<img src="../Fotos/foto01.png" alt="Drawing" style="width:950px;"/>

# Limpeza de Dados em um DataSet de Carros usados do Ebay

* Esse projeto será especialmente em português devido a um pedido de um amigo.
 

### <img src="../Fotos/foto02.png" alt="Drawing" style="width:20px; float:left"/> &nbsp; Dataset
O dataset original foi construído a partir do ebay alemão e foi disponibilizado no Kaggle [aqui](https://www.kaggle.com/orgesleka/used-cars-database/data), com 370.000 elementos distintos.
<br>No entanto será usado a versão do site [dataquest](https://www.dataquest.io), que se encontra na pasta Dados.
<br>O pessoal do site alterou o dataset com os seguintes objetivos:
1. Para que tenha os erros mais comuns de uma coleta de dados 'crua' da internet, como é usual de se ter após a realização de pesquisas com ferramentas, como por exemplo [Scrapy](https://scrapy.org/).
2. Ter seu tamanho reduzido para 50.000 elementos apenas, o que torna o processamento mais rápido, já que o intuito desse projeto é que mais pessoas possam baixá-lo e rodá-lo com facilidade em seus distintos ambientes com facilidade.

In [1]:
import numpy as np
import pandas as pd

In [2]:
#Sera usado o latin-1, pois o padrao UTF-8 nao foi suficiente, outro possivel encoding popular seria o Windows-1252
autos = pd.read_csv("../Dados/autos.csv", encoding = "latin-1") 

In [3]:
autos.shape

(50000, 20)

In [4]:
autos.head(5)

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [5]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

<b>Primeiras observações</b>
* Como primeira análise podemos perceber que, como dito anteriormente, esse dataset possui 50.000 elementos, sendo eles divididos em 20 colunas distintas.
* E podemos perceber também que algumas colunas que poderiam estar com tipos, numéricos por exemplo, como preço e quilometragem rodada que ajudariam em análises estão sem formatação.
* Algumas colunas possuem objetos nulos, porém a maior parte está completa.
* As colunas possuem formatação de nome em [cammelCase](https://pt.wikipedia.org/wiki/CamelCase), em vez do [snake_case](https://en.wikipedia.org/wiki/Snake_case) como se opta em boas práticas de python.


In [6]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [7]:
#Renomeando colunas
autos.rename({"dateCrawled":"crawled_date",
               "offerType":"offer_type",
               "vehicleType":"vehicle_type",
               "yearOfRegistration":"registration_year",
               "powerPS":"power_ps",
               "monthOfRegistration":"registration_month",
               "fuelType":"fuel_type",
               "notRepairedDamage":"damage",
               "dateCreated":"add_date",
               "nrOfPictures":"pictures_number",
               "postalCode":"postal_code",
               "lastSeen":"last_seen"}, axis=1, inplace=True)
print(autos.columns)

Index(['crawled_date', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand', 'damage',
       'add_date', 'pictures_number', 'postal_code', 'last_seen'],
      dtype='object')


In [8]:
autos.describe(include='all')

,crawled_date,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,damage,add_date,pictures_number,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-11 22:38:16,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Como pode ser percebido acima na descrição, existem colunas como preço, que são colunas chaves para se ter análises numéricas e é inviável no formato que estão.

### Transfomação numérica:

Inicialmente é de boa prática usar uma amostra aleatória do conteúdo que será modificado para analisar alguma possível anomalia:

In [9]:
autos["price"].sample(5)

17702       $500
44709       $690
26333     $3,999
21771     $1,690
15170    $15,500
Name: price, dtype: object

In [10]:
autos["price"] = autos["price"].str.replace("$","").str.replace(",","")
autos["price"] = autos["price"].astype(int)
autos["price"].dtype

dtype('int32')

Após a modificação é importante informar a unidade de medida da coluna, já que será modificado as referência nos elementos:

In [11]:
autos.rename({"price":"price_us"}, axis=1, inplace=True)

O mesmo procedimento, mas para outras colunas:

In [12]:
#Transformando em valores numericos a kilometragem
print(autos["odometer"].sample(3))

5665      70,000km
9140     150,000km
22082    150,000km
Name: odometer, dtype: object


In [13]:
autos["odometer"] = autos["odometer"].str.replace(",","").str.replace("km","")
autos["odometer"] = autos["odometer"].astype(int)

autos.rename({"odometer":"odometer_km"}, axis=1, inplace=True)

### Colunas com valores praticamente únicos:

Como pode ser percebido na descrição da DataFrame, as colunas 'seller' e 'offer_type', possuem 49999 dos 50000 elementos iguais, ou seja essas colunas não possuem valor a contribuir para análises, portanto podemos retirá-las do dataset

In [14]:
autos = autos.drop("seller", axis=1)
autos = autos.drop("offer_type", axis=1)

### Retirando pontos muito fora do padrão (outliers):

Um dos métodos mais simples para encontrar parte dos outliers é procurar dentro de dados numéricos por valores muito mais altos ou muito mais baixo que o padrão:

In [15]:
print("PREÇO:")
print("shape dos valores unicos: ", autos["price_us"].unique().shape)
print("\ndescricao:", autos["price_us"].describe())

PREÇO:
shape dos valores unicos:  (2357,)

descricao: count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price_us, dtype: float64


Pode-se perceber que o mínimo e o máximo estão muito fora de um padrão.<br>
Portanto os valores da coluna serão análisados com o intuito de encontrar porcentagens limitrofes que definirão os outliers.

In [16]:
for percentage in range(1, 10, 2):
    print("Percentage ", percentage, ": ", autos["price_us"].quantile(percentage/100))
print()
for percentage in range(30, 41, 2):
    print("Percentage ", percentage/10, ": ", autos["price_us"].quantile(percentage/1000))

Percentage  1 :  0.0
Percentage  3 :  1.0
Percentage  5 :  200.0
Percentage  7 :  300.0
Percentage  9 :  450.0

Percentage  3.0 :  1.0
Percentage  3.2 :  14.968000000000075
Percentage  3.4 :  64.83000000000061
Percentage  3.6 :  100.0
Percentage  3.8 :  111.0
Percentage  4.0 :  150.0


Logo pode-se notar que os preços com valores abaixo dos 3.2% (~15 dólares) estão muito fora de um padrão.

In [17]:
for percentage in range(90, 100, 2):
    print("Percentage ", percentage, ": ", autos["price_us"].quantile(percentage/100))

print()
for percentage in range(985, 1001, 5):
    print("Percentage ", percentage/10, ": ", autos["price_us"].quantile(percentage/1000))
    
print()
for values in autos["price_us"]:
    if values>1000000:
        print(values/1000000)

Percentage  90 :  14000.0
Percentage  92 :  15900.0
Percentage  94 :  18000.0
Percentage  96 :  21950.0
Percentage  98 :  28000.0

Percentage  98.5 :  31400.74999999997
Percentage  99.0 :  35900.0
Percentage  99.5 :  47000.0
Percentage  100.0 :  99999999.0

11.111111
1.3
10.0
1.234566
11.111111
12.345678
12.345678
99.999999
27.322222
12.345678
3.89


No caso do limite superior temos 6 exemplares entre 10 e 12 milhões, mas os últimos dois que são ~30milhões e ~100 milhões estão muito fora do padrão.

Com os limites inferiores e superiores definidos em preço, serão retiradas as respectivas linhas que estão fora desses limites

In [18]:
autos= autos[autos["price_us"].between(15.0,13000000.0)]

<br>
<br>
O mesmo processo será repetido para a <b>kilometragem</b>:

In [19]:
autos["odometer_km"].unique().shape

(13,)

Já que a kilometragem possui apenas 13 valores diferentes a análise é mais simples.

In [20]:
autos["odometer_km"].value_counts()

150000    31304
125000     5047
100000     2108
90000      1733
80000      1414
70000      1215
60000      1153
50000      1013
40000       815
5000        807
30000       779
20000       761
10000       249
Name: odometer_km, dtype: int64

Pode-se perceber que a kilometragem é padronizada pelo site e que não há valores máximos e mínimos suficientemente isolados que gerem a necessidade de serem retirados do dataset. 

In [21]:
autos.shape

(48398, 18)

Foram retiradas 1602 linhas do dataset original com as retirada dos outliers do preço. É um valor satisfatório já que a perda de dados não foi muito grande e com isso conseguiu-se alcançar uma qualidade melhor dos dados para análise.

### Analisando colunas com datas:

No banco existem cinco colunas representadas por datas.<br>
Dentre elas três foram criadas pelo _**site:**_ 
* "registration_month"
* "registration_year"
* "add_date".

Já as outras duas, foram criadas pelo _**crawler:**_
* "crawled_date"
* "last_seen".

In [22]:
autos[['crawled_date','add_date','last_seen', 'registration_month', 'registration_year']].dtypes

crawled_date          object
add_date              object
last_seen             object
registration_month     int64
registration_year      int64
dtype: object

É notório que as três primeiras colunas são strings e as últimas duas intergers.

In [23]:
autos[['crawled_date','add_date','last_seen','registration_month', 'registration_year']][0:5]

,crawled_date,add_date,last_seen,registration_month,registration_year
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54,3,2004
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08,6,1997
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37,7,2009
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28,6,2007
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50,7,2003


Será realizado um estudo sobre os valores encontrados nas colunas com elementos em string. Para isso será utilizado como base os dez primeiros caracteres, que representam o dia, a partir deles será possível realizar uma distribuição dos mesmos em porcentagem, o que permitirá uma análise dos padrões a partir dos valores encontrados.

In [24]:
autos['crawled_date'].str[:10].value_counts(normalize=True, dropna=False).sort_index

<bound method Series.sort_index of 2016-04-03    0.038576
2016-03-20    0.037770
2016-03-21    0.037316
2016-03-12    0.036964
2016-03-14    0.036634
2016-04-04    0.036530
2016-03-07    0.036035
2016-04-02    0.035497
2016-03-28    0.034857
2016-03-19    0.034754
2016-03-15    0.034258
2016-03-29    0.034154
2016-04-01    0.033741
2016-03-30    0.033741
2016-03-08    0.033286
2016-03-09    0.033059
2016-03-22    0.032956
2016-03-11    0.032584
2016-03-23    0.032274
2016-03-26    0.032253
2016-03-10    0.032212
2016-03-31    0.031840
2016-03-17    0.031613
2016-03-25    0.031551
2016-03-27    0.031096
2016-03-16    0.029485
2016-03-24    0.029402
2016-03-05    0.025352
2016-03-13    0.015641
2016-03-06    0.014050
2016-04-05    0.013079
2016-03-18    0.012893
2016-04-06    0.003161
2016-04-07    0.001384
Name: crawled_date, dtype: float64>

Com os dados resultantes da distribuição da coluna 'crawled_date' pode-se afirmar que a pesquisa que gerou os dados foi realizada aproximadamente em um periódo de um mês e meio, entre Março e Abril de 2016.

In [25]:
autos['add_date'].str[:10].value_counts(normalize=True, dropna=False).sort_index

<bound method Series.sort_index of 2016-04-03    0.038824
2016-03-20    0.037832
2016-03-21    0.037543
2016-04-04    0.036902
2016-03-12    0.036778
2016-03-14    0.035270
2016-04-02    0.035187
2016-03-28    0.034981
2016-03-07    0.034774
2016-03-29    0.034092
2016-03-15    0.033989
2016-04-01    0.033720
2016-03-19    0.033638
2016-03-30    0.033555
2016-03-08    0.033286
2016-03-09    0.033142
2016-03-11    0.032915
2016-03-22    0.032770
2016-03-26    0.032315
2016-03-23    0.032109
2016-03-10    0.031923
2016-03-31    0.031881
2016-03-25    0.031675
2016-03-17    0.031282
2016-03-27    0.031014
2016-03-16    0.029981
2016-03-24    0.029340
2016-03-05    0.022914
2016-03-13    0.017005
2016-03-06    0.015311
                ...   
2016-02-20    0.000041
2016-02-14    0.000041
2016-02-02    0.000041
2016-02-18    0.000041
2016-02-12    0.000041
2016-02-26    0.000041
2016-02-24    0.000041
2016-01-10    0.000041
2016-02-05    0.000041
2016-01-16    0.000021
2016-02-16    0.000021

É notório que a maior parte dos veículos no site foram anunciados no período da pesquisa ou logo antes.

In [26]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index

<bound method Series.sort_index of 2016-04-06    0.221621
2016-04-07    0.132010
2016-04-05    0.124985
2016-03-17    0.028059
2016-04-03    0.025187
2016-04-02    0.024877
2016-03-30    0.024753
2016-04-04    0.024505
2016-03-31    0.023865
2016-03-12    0.023782
2016-04-01    0.022873
2016-03-29    0.022336
2016-03-22    0.021385
2016-03-28    0.020869
2016-03-21    0.020600
2016-03-20    0.020600
2016-03-24    0.019753
2016-03-25    0.019195
2016-03-23    0.018596
2016-03-26    0.016798
2016-03-16    0.016447
2016-03-15    0.015868
2016-03-19    0.015827
2016-03-27    0.015600
2016-03-14    0.012624
2016-03-11    0.012377
2016-03-10    0.010620
2016-03-09    0.009649
2016-03-13    0.008864
2016-03-08    0.007356
2016-03-18    0.007314
2016-03-07    0.005413
2016-03-06    0.004318
2016-03-05    0.001074
Name: last_seen, dtype: float64>

Os dados acima demonstram uma clara anormalidade, pois em um período de apenas dois dias mais de 25% dos anúncios foram retirados. Esse fato pode demonstrar um improvável e atípico pico  nas vendas, ou, mais provavelmente, esses números estejam relacionados com o período de termino da pesquisa.

Também será analisada a coluna do ano de registro:

In [27]:
autos["registration_year"].describe()

count    48398.000000
mean      2004.773028
std         88.793368
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

Como pode ser visto acima, a coluna que representa o ano de registro dos veículos também possui anormalidades, como o ano mínimo sendo há mais de 10 séculos e o ano máximo estando no futuro.<br>
Portando serão definidos limites para as datas de registro. <br>
O valor máximo será 2016, pois é o ano da pesquisa, não há como ter carros registrados em anos futuro a esse.<br>
O valor mínimo será 1908, pois foi um ano que representa o começo da fabricação em massa de automóveis com o [Model T da ford](https://pt.wikipedia.org/wiki/Ford_Model_T).

### Tratando anomalias nos anos de registro:

Primeiramente será analisado qual a porcentagem dos dados que possui anomalias, esse valor será decisório para a escolha do formato de solução.

In [28]:
quantidade_de_valores_normais = autos["registration_year"].between(1908,2016).sum()
valor_total = autos.shape[0]
quantidade_de_valores_anormais = valor_total - quantidade_de_valores_normais

print("Porcentagem de valores anormais: ", quantidade_de_valores_anormais/valor_total)

Porcentagem de valores anormais:  0.03880325633290632


Como verificado acima as anomalias representam menos de 4% do dataset, logo as linhas que aprensetam-las serão deletadas.

In [29]:
autos = autos[autos["registration_year"].between(1908,2016)]

In [30]:
autos["registration_year"].value_counts(normalize=True)[:10]

2000    0.067175
2005    0.062898
1999    0.062059
2004    0.058018
2003    0.057932
2006    0.057416
2001    0.056599
2002    0.053289
1998    0.050537
2007    0.048882
Name: registration_year, dtype: float64

Como pode ser observado a maior parte dos carros foram registrados nos últimos vinte anos.

### Explorando marcas por preços

In [31]:
autos["brand"].value_counts(normalize=True)

volkswagen        0.211393
bmw               0.109974
opel              0.107330
mercedes_benz     0.096475
audi              0.086586
ford              0.069991
renault           0.047141
peugeot           0.029880
fiat              0.025688
seat              0.018250
skoda             0.016402
nissan            0.015305
mazda             0.015241
smart             0.014209
citroen           0.014037
toyota            0.012747
hyundai           0.010039
sonstige_autos    0.009673
volvo             0.009114
mini              0.008770
mitsubishi        0.008190
honda             0.007846
kia               0.007094
alfa_romeo        0.006642
porsche           0.006083
suzuki            0.005933
chevrolet         0.005696
chrysler          0.003525
dacia             0.002644
daihatsu          0.002515
jeep              0.002279
subaru            0.002128
land_rover        0.002107
saab              0.001655
jaguar            0.001569
daewoo            0.001483
trabant           0.001397
r

Como pode ser visto acima das cinco marcas com mais carros á venda, cinco são de montadoras alemãs. (Lembrando que esse dataset é originário do ebay alemão).
<br>Outro fator notório é que a Volkswagen possui um valor substancialmente maior que as outras marcas, chegando a quase o dobro da segunda colocada.
<br>Muitas das marcas acima possui porcentagens irrisórias em relação á quantidade total, então para análise dos preços por marca só serão adotadas as marcas que possuem mais de 2% de veículos em relação ao total.


In [32]:
contador_de_marcas = autos["brand"].value_counts(normalize=True)
marcas_comuns = contador_de_marcas[contador_de_marcas > 0.02].index
print(marcas_comuns)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat'],
      dtype='object')


In [33]:
preco_por_marca = {}

for marca in marcas_comuns:
    dados_das_linhas_da_marca = autos[autos["brand"]==marca]
    preco_medio = dados_das_linhas_da_marca["price_us"].mean()
    preco_por_marca[marca] = round(preco_medio)

preco_por_marca

{'volkswagen': 6751,
 'bmw': 8608,
 'opel': 5465,
 'mercedes_benz': 8657,
 'audi': 9367,
 'ford': 7477,
 'renault': 2484,
 'peugeot': 3101,
 'fiat': 2818}

Entende-se pelos dados acima 3 divisões de preço:
* Audi, Bmw e Mercedes-Benz são os mais caros;
* Renault, Peugeot e Fiat são os mais baratos;
* Volkswagen, Ford e Opel possuem valores médios.
<br><br>Algo chamativo é o fato que das três marcas mais populares, duas estão dentro do grupo de valores médios, incluindo a Volkswagen, a mais popular.

### Explorando marcas por kilometragem

Utilizando as mesmas marcas encontradas anteriormente (com o critério de acima de 2% do total de veículos), será estudado a kilometragem média pertecente a cada uma.

In [34]:
kilometragem_por_marca = {}

for marca in marcas_comuns:
    dados_das_linhas_da_marca = autos[autos["brand"]==marca]
    kilometragem_media = dados_das_linhas_da_marca["odometer_km"].mean()
    kilometragem_por_marca[marca] = round(kilometragem_media)
    
kilometragem_por_marca

{'volkswagen': 128723,
 'bmw': 132582,
 'opel': 129339,
 'mercedes_benz': 130839,
 'audi': 129234,
 'ford': 124244,
 'renault': 128144,
 'peugeot': 127122,
 'fiat': 117067}

Os dados acima mostram que a kilometragem média possui uma variação pequena entre as marcas.

### Agregando dados

Com as últimas duas análises (kilometragens e preços médios), gerou se novas informações que podem ser úteis em futuras análises, portanto esssas informações serão agragadas às informações atuais.

In [35]:
#Criando uma Serie com os dados dos precos medios por marcas
serie_marca_preco = pd.Series(preco_por_marca)
serie_marca_preco


volkswagen       6751
bmw              8608
opel             5465
mercedes_benz    8657
audi             9367
ford             7477
renault          2484
peugeot          3101
fiat             2818
dtype: int64

In [36]:
#Criando uma Serie com os dados das kilometragens medias por marca
serie_marca_km = pd.Series(kilometragem_por_marca)
serie_marca_km

volkswagen       128723
bmw              132582
opel             129339
mercedes_benz    130839
audi             129234
ford             124244
renault          128144
peugeot          127122
fiat             117067
dtype: int64

In [37]:
#Criando um dataframe a partir da series criada anteriormente
df_marca_preco_km = pd.DataFrame(serie_marca_preco, columns=['mean_price'])
#Adicionando Serie de kilometragem ao dataframe
df_marca_preco_km["mean_kilometrage"] = serie_marca_km

df_marca_preco_km

,mean_price,mean_kilometrage
volkswagen,6751,128723
bmw,8608,132582
opel,5465,129339
mercedes_benz,8657,130839
audi,9367,129234
ford,7477,124244
renault,2484,128144
peugeot,3101,127122
fiat,2818,117067


Com os dados lado a lado é possível notar uma kilometragem média menor em marcas que possuem menor preço médio (sendo essa variação muito pequena, já que a variação da kilometragem é pequena).

---
# <img src="../Fotos/foto03.png" alt="Drawing" style="width:30px; float:left"/> &nbsp; Salvando os dados limpos

Com as ações acima atingiu-se uma limpeza básica dos dados como proposta no projeto, é possível a utilização de mais técnicas, mas é de grande importância entender a necessidade do grau de limpeza que é necessário se obter.

Salvando os dados como um novo dataset limpo:

In [38]:
autos.to_csv('../Dados_limpos/autos_cleaned.csv',index=False)

Salvando as novas informações criadas com a análise gerada na limpeza:

In [39]:
df_marca_preco_km.to_csv('../Dados_limpos/autos_dados_adicionais.csv',index=False)